In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Thompson,CA,2020-10-23 05:46:25,55.74,-97.86,24.80,85,90,5.82
1,1,Alta Floresta,BR,2020-10-23 05:51:18,-9.88,-56.09,71.47,92,76,2.15
2,2,Hobart,AU,2020-10-23 05:48:13,-42.88,147.33,60.80,63,75,11.41
3,3,Mataura,NZ,2020-10-23 05:51:18,-46.19,168.86,59.00,70,99,5.01
4,4,Pangody,RU,2020-10-23 05:51:18,65.85,74.49,21.83,97,89,10.31


In [4]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [12]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig


Figure(layout=FigureLayout(height='420px'))

In [13]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [15]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
8,8,Cabo San Lucas,MX,2020-10-23 05:51:19,22.89,-109.91,80.01,82,0,9.71
10,10,Saint-Joseph,RE,2020-10-23 05:51:19,-21.37,55.62,80.60,54,0,17.22
11,11,Bengkulu,ID,2020-10-23 05:51:19,-3.80,102.27,82.63,70,100,7.14
14,14,Nelson Bay,AU,2020-10-23 05:51:20,-32.72,152.15,79.00,57,16,14.99
15,15,Saint-Philippe,RE,2020-10-23 05:51:20,-21.36,55.77,80.60,54,0,17.22
17,17,Bambous Virieux,MU,2020-10-23 05:49:10,-20.34,57.76,79.00,65,75,14.99
28,28,Atuona,PF,2020-10-23 05:51:22,-9.80,-139.03,78.15,71,0,13.40
29,29,Tomatlan,MX,2020-10-23 05:51:22,19.93,-105.25,77.38,95,57,1.10
30,30,Hobyo,SO,2020-10-23 05:51:22,5.35,48.53,84.00,70,100,10.04
32,32,Ramganj Mandi,IN,2020-10-23 05:51:23,24.65,75.93,89.35,15,0,3.44


In [16]:
preferred_cities_df.count()

City_ID       170
City          170
Country       170
Date          170
Lat           170
Lng           170
Max Temp      170
Humidity      170
Cloudiness    170
Wind Speed    170
dtype: int64

In [17]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
8,Cabo San Lucas,MX,80.01,22.89,-109.91,
10,Saint-Joseph,RE,80.60,-21.37,55.62,
11,Bengkulu,ID,82.63,-3.80,102.27,
14,Nelson Bay,AU,79.00,-32.72,152.15,
15,Saint-Philippe,RE,80.60,-21.36,55.77,
17,Bambous Virieux,MU,79.00,-20.34,57.76,
28,Atuona,PF,78.15,-9.80,-139.03,
29,Tomatlan,MX,77.38,19.93,-105.25,
30,Hobyo,SO,84.00,5.35,48.53,
32,Ramganj Mandi,IN,89.35,24.65,75.93,


In [18]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [24]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
    print("Hotel not found... skipping.")

In [25]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
8,Cabo San Lucas,MX,80.01,22.89,-109.91,Hotel Tesoro Los Cabos
10,Saint-Joseph,RE,80.60,-21.37,55.62,"""Plantation Bed and Breakfast"
11,Bengkulu,ID,82.63,-3.80,102.27,Grage Hotel Bengkulu
14,Nelson Bay,AU,79.00,-32.72,152.15,Mantra Nelson Bay
15,Saint-Philippe,RE,80.60,-21.36,55.77,"Chambres d'hôte ""La Trinité"""
...,...,...,...,...,...,...
561,Camapua,BR,77.65,-19.53,-54.04,
563,Hermosillo,MX,77.00,29.07,-110.97,
566,Meulaboh,ID,86.86,4.14,96.13,
567,Coahuayana,MX,78.96,18.73,-103.68,


In [26]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [27]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [35]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [36]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [37]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [38]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))